### Install numpy version 1.19.3 

In [12]:
#vesrion 1.19.4 giving errors
#!pip install numpy==1.19.3

ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.
You should consider upgrading via the 'C:\Users\christine.njuguna\opencv\Scripts\python.exe -m pip install --upgrade pip' command.


In [37]:
import numpy as np
import cv2

### toBin to convert all int values to binary

In [38]:
def toBin(data):
    #Convert `data` to binary format as string
    if type(data)== str:
        return ''.join([ format(ord(i), "08b") for i in data ])
    elif type(data)== bytes or type(data)== np.ndarray:
        return [ format(i, "08b") for i in data ] 
    elif type(data)== int or type(data) == np.uint8:
        return format(data, "08b")
    else:
        raise TypeError("Type not supported.")

### encode the message into the image

In [40]:
def encode(image_name, secret_data):
    
    # read the image
    image = cv2.imread(image_name)
    
    # maximum bytes to encode
    n_bytes = image.shape[0] * image.shape[1] * 3 // 8
    print("[*] Maximum bytes to encode:", n_bytes)
    
    #check to see if the image size is enough to encode the message.
    if len(secret_data) > n_bytes:
        raise ValueError("[!] Insufficient bytes, need bigger image or less data.")
    print("[*] Encoding data...")
    
    
    # add stopping criteria so that while decoding you know where to stop
    secret_data += "====="
    data_index = 0
    
    
    # convert secret message to binary
    binary_secret_data = toBin(secret_data)
    
    # size of data to hide
    data_len = len(binary_secret_data)
    
    #Loop through each row and each pixel in each row
    for row in image:
        for pixel in row:
            
            # each pixel has an RGB value which needs to be converted to binary
            r, g, b = toBin(pixel)
            
            # modify the least significant bit only if there is still data to store
            if data_index < data_len:
                # least significant red pixel bit
                pixel[0] = int(r[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
                
            if data_index < data_len:
                # least significant green pixel bit
                pixel[1] = int(g[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
                
            if data_index < data_len:
                # least significant blue pixel bit
                pixel[2] = int(b[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
                
            # if all the data is encoded exit the loop
            if data_index >= data_len:
                break
                
    return image

### Decode the image and retrieve the hidden message

In [41]:
def decode(image_name):
    print("[+] Decoding...")
    
    # read the image
    image = cv2.imread(image_name)
    binary_data = ""
    
    #Loop through each row and each pixel in each row
    for row in image:
        for pixel in row:
            r, g, b = toBin(pixel)
            binary_data += r[-1]
            binary_data += g[-1]
            binary_data += b[-1]
            
    # split the continuous string of bits into a list of bytes
    all_bytes = [ binary_data[i: i+8] for i in range(0, len(binary_data), 8) ]
    
    #print(all_bytes)
    
    # convert from bits to characters
    decoded_data = ""
    
    for byte in all_bytes:
        # chr() function converts int to unicode
        decoded_data += chr(int(byte, 2))
        
        #when you arrive at the delimiter stop decoding
        if decoded_data[-5:] == "=====":
            break
            
    return decoded_data[:-5]

In [ ]:
#Main method to tun the encode and decode modules
input_image = "image.PNG"
output_image = "encoded_image.PNG"
secret_data = "This is a top secret message."

# start encoding
encoded_image = encode(image_name=input_image, secret_data=secret_data)

# save the output image (encoded image)
cv2.imwrite(output_image, encoded_image)

# start decoding
decoded_data = decode(output_image)
print("[+] Decoded data:", decoded_data)